following block is for generating the local cluster expansion model. See the Table S3 in support information of nasicon kmc as reference

The localclusterexpansion.initialization is for generating the **lce.json and lcesite.json**

the kmc.event_generator is for generating the **event_kernal.csv and events.json**

read the primitive cell from prim.json, and write events.json as output

In [ ]:
from kmcpy.event_generator import generate_events

generate_events(prim_fname=r"../examples/inputs/prim.json",supercell_shape=[2,1,1],event_fname="./events.json",event_kernal_fname='./event_kernal.csv')

Api version 2 initiated
aiming for more flexible inputs.
Following block is the main KMC function
DO make sure that they provide similar result as in API=1
KMC result is not necessarily the same, since it is randomized

In [ ]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=2,**kwargs):
    inputset=InputSet.from_json("test_input_v2.json",api=2)

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    inputset.set_parameter("occ",load_occ(fname=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
    
    
    

    # step 1 initialize global occupation and conditions
    
    
    
    kmc = KMC(api=api)
    events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

    # # step 2 compute the site kernal (used for kmc run)
    kmc.load_site_event_list(inputset._parameters["event_kernel"])

    # # step 3 run kmc
    kmc.run_from_database(events=events_initialized,**inputset._parameters)


if __name__ == "__main__":

    main()

testing the sites_and_lattices.py

我写的site lattice看不懂.


根据piero建议,还是用structure写吧.....

In [ ]:
from kmcpy.external.pymatgen_structure import Structure
from pymatgen.symmetry.analyzer  import SpacegroupAnalyzer
from kmcpy.external.pymatgen_local_env import CutOffDictNN
structure=Structure.from_cif("/Users/weihangxie/Documents/GitHub/kmcPy/dev_test/EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")



#print(structure.site_properties)
print(structure)

#print(structure[5].properties)

structure.make_supercell(scaling_matrix=[1,1,2])
#print(structure.site_properties)
local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4}
local_env_finder = CutOffDictNN(local_env_cutoff_dict)
local_env_finder.get_nn_info(structure,0)   
from kmcpy.model import LocalClusterExpansion



a=LocalClusterExpansion(api=1)
a.initialization(center_mobile_ion_specie_1_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a=LocalClusterExpansion(api=2)
a.initialization(mobile_ion_specie_1_index_index=0,cutoff_cluster=[6,6,6],cutoff_region=4,template_cif_fname='./EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif')
a.to_json("test_LCEv2.json")


test on find_site_by_wyckoff_sequence_and_label

test on nd supercell

test the new event generator

Problem!!!!! LOok the distance matrix, they are not all the same

Fixed now

In [2]:
from kmcpy.event_generator import generate_events2
from kmcpy.external.pymatgen_structure import Structure

primitive_structure=Structure.from_file("EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",primitive=True)
primitive_structure.to("cif","primitive_EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif")


#generate_events2(prim_cif_name="primitive_EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,3,4],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',mobile_ion_specie_1_index_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True)


#generate_events2(prim_cif_name="EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,3,4],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',mobile_ion_specie_1_index_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True,hacking_arg={1: [27, 29, 28, 30, 32, 31, 117, 119, 118, 120, 122, 121],2:[21,22,23,32,30,31,111,112,113,122,120,121],3:[18,20,19,34,33,35,108,110,109,124,123,125],5:[21,23,22,24,26,25,111,113,112,114,116,115]})
#generate_events2(prim_cif_name="EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,1,1],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',mobile_ion_specie_1_index_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True,hacking_arg={2:[21,22,23,32,30,31,111,112,113,122,120,121],3:[18,20,19,34,33,35,108,110,109,124,123,125],5:[21,23,22,24,26,25,111,113,112,114,116,115]})

generate_events2(prim_cif_name="EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif",supercell_shape=[2,1,1],local_env_cutoff_dict = {('Na+','Na+'):4,('Na+','Si4+'):4},event_fname="events.json",event_kernal_fname='event_kernal.csv',mobile_ion_specie_1_index_label_or_indices="Na1",species_to_be_removed=['Zr4+','O2-','O','Zr'],verbose=True,hacking_arg=None,convert_to_primitive_cell=False,export_reference_cluster="reference_cluster.cif")




FileNotFoundError: [Errno 2] No such file or directory: 'EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif'

完整的第二版测试?

注意,prim.json读进structure之后,再输出成cif,注意输出cif时候要给过去一个symprec变量.这样生成出来的cif是有对称性的.然后要参照cif里面的Label更改center atom 和 diffuse to atom. 这里面分别是Na1 和Na0

20220518 已经移动到unitttest


20220519 start 3rd version on grain boundary model